In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy import stats
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, SVD, NMF, accuracy
from collections import defaultdict

# Data Parsing

In [22]:
# initialize data
item_threshold = 1 # used to filter out user/artist pairs that have been 
                   #listened to less than the threshold number of times
popular_artist_fraction = 0.2 # top cutoff for what we consider popular artists, in this case the top 20%

user_events_file = 'data/user_events.txt'
low_user_file = 'data/low_main_users.txt'
medium_user_file = 'data/medium_main_users.txt'
high_user_file = 'data/high_main_users.txt'

In [23]:
#read in user events file
cols = ['user', 'artist', 'album', 'track', 'timestamp']
df_events = pd.read_csv(user_events_file, sep='\t', names=cols)
print('No. of user events: ' + str(len(df_events)))
df_events.head() # check it is all read in properly

No. of user events: 28718087


,user,artist,album,track,timestamp
0,31435741,2,4,4,1385212958
1,31435741,2,4,4,1385212642
2,31435741,2,4,4,1385212325
3,31435741,2,4,4,1385209508
4,31435741,2,4,4,1385209191


## User-Artist Matrix

In [24]:
# create unique user-artist matrix
df_events = df_events.groupby(['user', 'artist']).size().reset_index(name='listens')
print('No. user-artist pairs: ' + str(len(df_events)))
# each row contains a unique user-artist pair, along with how many times the
# user has listened to the artist
df_events.head()

No. user-artist pairs: 1755361


,user,artist,listens
0,1021445,12,43
1,1021445,16,1
2,1021445,28,7
3,1021445,29,1
4,1021445,46,1


In [25]:
# filters out artist/user pairs who havent been listened two more than
# item_threshold amount of times to reduce
# kept mostly to 1 so we dont filter out any data currently
df_events = df_events[df_events['listens'] >= item_threshold] 

# With 1, we see no difference between user-artist pairs here
print('No. filtered user-artist pairs: ' + str(len(df_events))) 

# here, we see the number of unique artists in our matrix
print('No. unique artists: ' + str(len(df_events['artist'].unique())))

No. filtered user-artist pairs: 1755361
No. unique artists: 352805


#### How many artists have users listened to?

In [26]:
# get matrix where each row is a user-id and how many artists they've 
#listened to
user_dist = df_events['user'].value_counts() 

# counts how many unique users there are. prints out user id & a count of how 
# many rows they're included in, which effectively shows how many artists 
# they listen to
num_users = len(user_dist)
print('Mean artists of all users: ' + str(user_dist.mean()))
print('Min artists of all users: ' + str(user_dist.min()))
print('Max artists of all users: ' + str(user_dist.max()))

user_dist.head()

Mean artists of all users: 585.1203333333333
Min artists of all users: 18
Max artists of all users: 4011


41888522    4011
4393555     3700
40029632    3678
26874346    3544
29736410    3529
Name: user, dtype: int64

#### How many users listen to an artist?

In [27]:
# get artist distribution
# same as previous but with artists, shows artist-id and how many times they
# were listened to buy unique users
artist_dist = df_events['artist'].value_counts()
num_artists = len(artist_dist)
print('No. artists: ' + str(num_artists))
df_events['artist'].value_counts().head

No. artists: 352805


<bound method NDFrame.head of 135        1389
1602       1359
46         1325
320        1297
27         1290
           ... 
3124286       1
1029181       1
1023032       1
1008679       1
3087545       1
Name: artist, Length: 352805, dtype: int64>

In [28]:
# get number of  popular artists
num_top_artists = int(popular_artist_fraction * num_artists)

# getting the top top_fraction (0.2) percent of artists, so finding how many
# artists make up 20% of total artists, and then only using the artists those
#number of the most popular aritsts
top_artist_dist = artist_dist[:num_top_artists]
print('No. top artists: ' + str(len(top_artist_dist)))

No. top artists: 70561


In [29]:
# read in users
# user file is just user_id and their mainstreaminess value 
low_users = pd.read_csv(low_user_file, sep=',').set_index('user_id')
medium_users = pd.read_csv(medium_user_file, sep=',').set_index('user_id')
high_users = pd.read_csv(high_user_file, sep=',').set_index('user_id')
num_users = len(low_users) + len(medium_users) + len(high_users)
print('Num users: ' + str(num_users))

Num users: 3000


## Calculating GAP of User Profiles

In [30]:
# get popularity metrics
pop_count = [] # number of top items per user
user_hist = [] # user history sizes
pop_fraq = [] # relative number of top items per user
pop_artist_fraq = [] # average popularity of items in user profiles
low_profile_size = 0
low_gap_p = 0
medium_profile_size = 0
medium_gap_p = 0
high_profile_size = 0
high_gap_p = 0
low_count = 0
med_count = 0
high_count = 0
for u, df in df_events.groupby('user'):
    no_user_artists = len(set(df['artist'])) # profile size //number of artists in users profile
    # top items in profile //top percent of pop artists in users profile
    no_user_pop_artists = len(set(df['artist']) & set(top_artist_dist.index)) 
    pop_count.append(no_user_pop_artists) # adds users # of pop artists to a list 
    user_hist.append(no_user_artists) #sizes of users listening histories
    pop_fraq.append(no_user_pop_artists / no_user_artists) #fraction of pop items in profile / total items
    # get popularity (= fraction of users interacted with item) of user items and calculate average of it
    user_pop_artist_fraq = sum(artist_dist[df['artist']] / no_users) / no_user_artists
    pop_artist_fraq.append(user_pop_artist_fraq)
    if u in low_users.index: # get user group-specific values
        low_profile_size += no_user_artists
        low_gap_p += user_pop_artist_fraq
        low_count += 1
    elif u in medium_users.index:
        medium_profile_size += no_user_artists
        medium_gap_p += user_pop_artist_fraq
        med_count += 1
    else:
        high_profile_size += no_user_artists
        high_gap_p += user_pop_artist_fraq
        high_count += 1
low_profile_size /= len(low_users)
medium_profile_size /= len(medium_users)
high_profile_size /= len(high_users)
low_gap_p /= len(low_users) # average popularity of items/artists in low/med/high groups (gap = group average popularity)
medium_gap_p /= len(medium_users)
high_gap_p /= len(high_users)
print('Low count (for check): ' + str(low_count))
print('Med count (for check): ' + str(med_count))
print('High count (for check): ' + str(high_count))

Low count (for check): 1000
Med count (for check): 1000
High count (for check): 1000


### Min-Max Scaling Ratings

In [31]:
### Scale listening counts on a scale from 1-1000
scaled_df_events = pd.DataFrame()
for user_id, group in df_events.groupby('user'):
    #print(group)
    min_listens = group['listens'].min()
    max_listens = group['listens'].max()
    std = (group['listens'] - min_listens) / (max_listens - min_listens)
    scaled_listens = std * 999 + 1
    to_replace = group.copy()
    to_replace['listens'] = scaled_listens
    #print(to_replace)
    scaled_df_events = scaled_df_events.append(to_replace)
scaled_df_events.head()   
#df_events.groupby('user').head()

,user,artist,listens
0,1021445,12,184.222707
1,1021445,16,1.000000
2,1021445,28,27.174672
3,1021445,29,1.000000
4,1021445,46,1.000000


# Analysis of Cutting Edge RecSys Algorithms